In [1]:
pip install overpy

In [2]:
import overpy
import requests
import json
import geojson
import pandas as pd
import numpy as np

In [3]:
def overpass_output(dist=5000, lat=53.340483, lng=-6.257701):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json][timeout:25];
    (
     node["amenity"="charging_station"](around:{dist},{lat},{lng});
    );
    out geom;
    """
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    response.raise_for_status()
    return response.json()

In [4]:
# Converting json file to dataframe - implement this piece
data_check = overpass_output()['elements']
data_check
df_ch = pd.json_normalize(data_check)
df_ch.head()

,type,id,lat,lon,tags.addr:street,tags.amenity,tags.amperage,tags.capacity,tags.fee,tags.name,...,tags.socket:chademo,tags.socket:chademo:output,tags.socket:type2_combo,tags.socket:type2_combo:output,tags.authentication:none,tags.location,tags.note,tags.payment:contactless,tags.level,tags.covered
0,node,2337216977,53.369002,-6.276171,Finglas Road,charging_station,16,2,yes,ESB Electric Vehicle Charge Point,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,2531423263,53.345114,-6.251316,Mark Street,charging_station,NaN,2,yes,ESB Electric Vehicle Charge Point,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,3664754602,53.347053,-6.299131,NaN,charging_station,NaN,2,yes,ESB Electric Vehicle Charge Point,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,4114614922,53.342752,-6.228181,Thorncastle Street,charging_station,NaN,2,yes,ESB Electric Vehicle Charge Point,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,4366316733,53.335304,-6.257683,NaN,charging_station,NaN,2,yes,ESB Electric Vehicle Charge Point,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_ch.columns

Index(['type', 'id', 'lat', 'lon', 'tags.addr:street', 'tags.amenity',
       'tags.amperage', 'tags.capacity', 'tags.fee', 'tags.name',
       'tags.operator', 'tags.operator:wikidata', 'tags.operator:wikipedia',
       'tags.ref', 'tags.socket:type2', 'tags.socket:type2:output',
       'tags.voltage', 'tags.addr:city', 'tags.addr:housename',
       'tags.opening_hours', 'tags.phone', 'tags.authentication:app',
       'tags.authentication:contactless', 'tags.motorcar',
       'tags.charging_station:capacity', 'tags.access', 'tags.parking',
       'tags.brand', 'tags.brand:wikidata', 'tags.website', 'tags.layer',
       'tags.source', 'tags.socket:chademo', 'tags.socket:chademo:output',
       'tags.socket:type2_combo', 'tags.socket:type2_combo:output',
       'tags.authentication:none', 'tags.location', 'tags.note',
       'tags.payment:contactless', 'tags.level', 'tags.covered'],
      dtype='object')

In [6]:
df_ch.dtypes

type                                object
id                                   int64
lat                                float64
lon                                float64
tags.addr:street                    object
tags.amenity                        object
tags.amperage                       object
tags.capacity                       object
tags.fee                            object
tags.name                           object
tags.operator                       object
tags.operator:wikidata              object
tags.operator:wikipedia             object
tags.ref                            object
tags.socket:type2                   object
tags.socket:type2:output            object
tags.voltage                        object
tags.addr:city                      object
tags.addr:housename                 object
tags.opening_hours                  object
tags.phone                          object
tags.authentication:app             object
tags.authentication:contactless     object
tags.motorc

In [7]:
# Review the outputs
df_summ = (
    df_ch
    .assign(capacity=lambda df_:df_['tags.capacity'].astype(float))
    .groupby(['tags.name', 'tags.access'], dropna=False)
    .agg({
        'capacity': ['sum', 'min', 'max', 'mean','count']
    })
)
df_summ

capacity              \
                                                        sum   min   max   
tags.name                              tags.access                        
EC Charging Fast Charger               permissive       2.0   2.0   2.0   
ESB Electric Vehicle Charge Point      customers        2.0   2.0   2.0   
                                       NaN             54.0   2.0   2.0   
ESB Electric Vehicle Fast Charge Point NaN             10.0   2.0   4.0   
EV Charging Stations                   permissive       6.0   6.0   6.0   
EasyGo Fast Charger                    NaN              1.0   1.0   1.0   
Electric Car Charging                  private          6.0   6.0   6.0   
LiFe EV                                permissive       2.0   2.0   2.0   
Lidl Electric Vehicle Charger          NaN              6.0   2.0   2.0   
ecar Charge Point                      customers        2.0   1.0   1.0   
flowbird park & charge                 NaN             10.0  10.0  10.0   
NaN                                    permissive       2.0   2.0   2.0   
                                       private          1.0   1.0   1.0   
                                       yes              2.0   2.0   2.0   
                                       NaN              2.0   2.0   2.0   

                                                                     
                                                         mean count  
tags.name                              tags.access                   
EC Charging Fast Charger               permissive    2.000000     1  
ESB Electric Vehicle Charge Point      customers     2.000000     1  
                                       NaN           2.000000    27  
ESB Electric Vehicle Fast Charge Point NaN           3.333333     3  
EV Charging Stations                   permissive    6.000000     1  
EasyGo Fast Charger                    NaN           1.000000     1  
Electric Car Charging                  private       6.000000     1  
LiFe EV                                permissive    2.000000     1  
Lidl Electric Vehicle Charger          NaN           2.000000     3  
ecar Charge Point                      customers     1.000000     2  
flowbird park & charge                 NaN          10.000000     1  
NaN                                    permissive    2.000000     1  
                                       private       1.000000     1  
                                       yes           2.000000     1  
                                       NaN           2.000000     1

In [8]:
data = overpass_output()
data

{'version': 0.6,
 'generator': 'Overpass API 0.7.61.5 4133829e',
 'osm3s': {'timestamp_osm_base': '2023-09-12T13:08:29Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'node',
   'id': 2337216977,
   'lat': 53.369002,
   'lon': -6.2761714,
   'tags': {'addr:street': 'Finglas Road',
    'amenity': 'charging_station',
    'amperage': '16',
    'capacity': '2',
    'fee': 'yes',
    'name': 'ESB Electric Vehicle Charge Point',
    'operator': 'ESB ecars',
    'operator:wikidata': 'Q3050566',
    'operator:wikipedia': 'en:ESB Group',
    'ref': 'C76VM',
    'socket:type2': '2',
    'socket:type2:output': '22kW',
    'voltage': '220'}},
  {'type': 'node',
   'id': 2531423263,
   'lat': 53.3451138,
   'lon': -6.2513165,
   'tags': {'addr:city': 'Dublin',
    'addr:street': 'Mark Street',
    'amenity': 'charging_station',
    'capacity': '2',
    'fee': 'yes',
    'name': 'ESB Electric Vehicl

### OLD VERSION

Code below used to assess the json output file. Retained for peer review.

In [9]:
# Create DataFrame for review
def create_df(input_dict=data.get('elements')):
    df = pd.DataFrame.from_dict(input_dict)
    # review tags list
    list_ch = df.tags.values.tolist()
    df2 = pd.DataFrame(list_ch)
    df3 = pd.concat([df, df2], axis=1)
    return (
        df3
        .assign(capacity=lambda df_:df_.capacity.astype(float))
        .drop(columns=['tags'])
    )

In [10]:
# Convert the feature data types to allow for correct calculations
df = create_df()
df.head()

,type,id,lat,lon,addr:street,amenity,amperage,capacity,fee,name,...,socket:chademo,socket:chademo:output,socket:type2_combo,socket:type2_combo:output,authentication:none,location,note,payment:contactless,level,covered
0,node,2337216977,53.369002,-6.276171,Finglas Road,charging_station,16,2.0,yes,ESB Electric Vehicle Charge Point,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,2531423263,53.345114,-6.251316,Mark Street,charging_station,NaN,2.0,yes,ESB Electric Vehicle Charge Point,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,3664754602,53.347053,-6.299131,NaN,charging_station,NaN,2.0,yes,ESB Electric Vehicle Charge Point,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,4114614922,53.342752,-6.228181,Thorncastle Street,charging_station,NaN,2.0,yes,ESB Electric Vehicle Charge Point,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,4366316733,53.335304,-6.257683,NaN,charging_station,NaN,2.0,yes,ESB Electric Vehicle Charge Point,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Review the outputs
df_summ = (
    df
    .groupby(['name', 'access'], dropna=False)
    .agg({
        'capacity': ['sum', 'min', 'max', 'mean','count']
    })
)
df_summ

capacity              \
                                                       sum   min   max   
name                                   access                            
EC Charging Fast Charger               permissive      2.0   2.0   2.0   
ESB Electric Vehicle Charge Point      customers       2.0   2.0   2.0   
                                       NaN            54.0   2.0   2.0   
ESB Electric Vehicle Fast Charge Point NaN            10.0   2.0   4.0   
EV Charging Stations                   permissive      6.0   6.0   6.0   
EasyGo Fast Charger                    NaN             1.0   1.0   1.0   
Electric Car Charging                  private         6.0   6.0   6.0   
LiFe EV                                permissive      2.0   2.0   2.0   
Lidl Electric Vehicle Charger          NaN             6.0   2.0   2.0   
ecar Charge Point                      customers       2.0   1.0   1.0   
flowbird park & charge                 NaN            10.0  10.0  10.0   
NaN                                    permissive      2.0   2.0   2.0   
                                       private         1.0   1.0   1.0   
                                       yes             2.0   2.0   2.0   
                                       NaN             2.0   2.0   2.0   

                                                                    
                                                        mean count  
name                                   access                       
EC Charging Fast Charger               permissive   2.000000     1  
ESB Electric Vehicle Charge Point      customers    2.000000     1  
                                       NaN          2.000000    27  
ESB Electric Vehicle Fast Charge Point NaN          3.333333     3  
EV Charging Stations                   permissive   6.000000     1  
EasyGo Fast Charger                    NaN          1.000000     1  
Electric Car Charging                  private      6.000000     1  
LiFe EV                                permissive   2.000000     1  
Lidl Electric Vehicle Charger          NaN          2.000000     3  
ecar Charge Point                      customers    1.000000     2  
flowbird park & charge                 NaN         10.000000     1  
NaN                                    permissive   2.000000     1  
                                       private      1.000000     1  
                                       yes          2.000000     1  
                                       NaN          2.000000     1